In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, accuracy_score,f1_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import Pipeline
from nltk.corpus import names
from collections import defaultdict
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

%matplotlib inline

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
data = pd.Series(newsgroups.data)
df = pd.DataFrame(data)
df.columns = ['text'] + df.columns.tolist()[1:]
df['target'] = pd.Series(newsgroups.target)
df.head()

,text,target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4


In [3]:
print('shape of dataframe: ', df.shape)
print('number of target variables: ',df.target.nunique())
print('null target variables: ', df.target.isna().any())
print('null text: ',df.text.isna().any())

shape of dataframe:  (18846, 2)
number of target variables:  20
null target variables:  False
null text:  False


In [4]:
#remove emails
df['text'] = df['text'].apply(lambda row: re.sub(r"\S*@\S*\s?","",row))

#remove single quote marks
df['text'] = df['text'].apply(lambda row: re.sub(r"\'","",row))

#make all text lower case
df['text'] = df['text'].str.lower()

#remove numerics
df['text'] = df['text'].apply(lambda row: re.sub(r"\d+","",row))

#remove words with fewer than 3 characters
df['text'] = df['text'].apply(lambda row: re.sub(r'\b\w{1,2}\b', '',row))

#remove extra spaces
df['text'] = df['text'].apply(lambda row: re.sub(r"\s+"," ",row))

df

,text,target
0,sure some bashers pens fans are pretty confus...,10
1,brother the market for high-performance video...,3
2,finally you said what you dream about. medite...,17
3,think! its the scsi card doing the dma transf...,3
4,) have old jasmine drive which cannot use with...,4
...,...,...
18841,> from: (david nye) > neurology > consultation...,13
18842,not isolated ground recepticles (usually unus...,12
18843,"just installed - cpu clone motherboard, and t...",3
18844,"wouldnt this require hyper-sphere. -space, po...",1


In [5]:
#instatiate tokenizer
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')

#tokenize test
df['tokenized_text'] = df.apply(lambda row: tokenizer.tokenize(row['text']),axis=1)

#define stop words
stop_words=set(stopwords.words("english"))

#remove stop words
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [item for item in x if item not in stop_words])

#instantiate lemmatizer
lemmatizer = WordNetLemmatizer()

#lemmatize text
df['lemmatized'] = df['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])
df.lemmatized = df.lemmatized.apply(lambda x: " ".join(x) )

df.sample(5)

,text,target,tokenized_text,lemmatized
3612,predict that the outcome the study what went ...,16,"[predict, outcome, study, went, wrong, federal...",predict outcome study went wrong federal assau...
15402,where did you hear this? printed book somewhe...,5,"[hear, printed, book, somewhere, throw, away, ...",hear printed book somewhere throw away book ac...
12785,"xperts, some simple questions for you: ive se...",5,"[xperts, simple, questions, ive, seen, lot, di...",xperts simple question ive seen lot different ...
9346,"you remember game the nlcs, after the dodgers...",9,"[remember, game, nlcs, dodgers, defeated, mets...",remember game nlcs dodger defeated mets ugh du...
2915,"big for .sig? way! keith "" home the billdboar...",0,"[big, sig, way, keith, home, billdboard, sig, ...",big sig way keith home billdboard sig file rya...


In [6]:
df.tokenized_text.isna().any()

False

In [7]:
#train test validation split

X,y = df.lemmatized,df.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [8]:
#pipeline for logistic regression

pipe_logreg = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

parameters_logreg = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5,1],
#     'tfidf__max_features': [None, 5000, 10000, 50000],
    'tfidf__ngram_range': [(1,1),(1, 2)],  # unigrams or bigrams
#     "clf__C": [0.1,1,10],
    "clf__class_weight": ['balanced'],
    "clf__solver": ['newton-cg','lbfgs'],
    "clf__max_iter":[80,100],
    "clf__multi_class":['multinomial','ovr']
}

gs_logreg = GridSearchCV(estimator=pipe_logreg,
            param_grid=parameters_logreg,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_logreg.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_logreg.best_score_)

# Best params
print('\nBest params:\n', gs_logreg.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  2.2min finished


Best accuracy: 0.702

Best params:
 {'clf__class_weight': 'balanced', 'clf__max_iter': 80, 'clf__multi_class': 'ovr', 'clf__solver': 'lbfgs', 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [9]:
#pipeline for decision tree

pipe_tree = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])
parameters_tree = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [0.75],
    'tfidf__ngram_range': [(1, 1)],  # unigrams or bigrams
    "clf__criterion": ["gini"],
    "clf__max_depth":[150,250],
    "clf__min_samples_split" : [200]
}

gs_tree = GridSearchCV(estimator=pipe_tree,
            param_grid=parameters_tree,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_tree.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_tree.best_score_)

# Best params
print('\nBest params:\n', gs_tree.best_params_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    7.1s remaining:    4.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   11.1s finished


Best accuracy: 0.481

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 250, 'clf__min_samples_split': 200, 'tfidf__max_df': 0.75, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 1)}


In [10]:
#pipeline for random forest

# pipe_rf = Pipeline([
#      ('tfidf', TfidfVectorizer()),
#     ('clf', RandomForestClassifier())
# ])
# parameters_rf = {
#     'tfidf__min_df': [3,5,10],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
#     'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
#     "clf__criterion": ["gini", "entropy"],
#     "clf__max_depth":[75,100,150],
#     "clf__min_samples_split" : [75,100,150]
# }

# gs_rf = GridSearchCV(estimator=pipe_rf,
#             param_grid=parameters_rf,
#             scoring='accuracy',
#             cv=KFold(5,shuffle=True,random_state=42), 
#             return_train_score = True, verbose=1,n_jobs=-1)

# # Fit using grid search
# best_model = gs_rf.fit(X_train, y_train)

# # Best accuracy
# print('Best accuracy: %.3f' % gs_rf.best_score_)

# # Best params
# print('\nBest params:\n', gs_rf.best_params_)

In [11]:
#pipeline for NaiveBayes

pipe_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   GaussianNB()),
])

parameters_nb = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 1), (1, 2)],  # unigrams or bigrams
    "svd__n_components": [100,150,250,500],
}

gs_nb = GridSearchCV(estimator=pipe_nb,
            param_grid=parameters_nb,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_nb.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_nb.best_score_)

# Best params
print('\nBest params:\n', gs_nb.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.1min finished


Best accuracy: 0.530

Best params:
 {'svd__n_components': 150, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [12]:
!pip install xgboost

In [13]:
#pipeline for svm
from sklearn.svm import SVC
pipe_svm = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',   SVC()),
])

parameters_svm = {
    'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1, 2)],  # unigrams or bigrams
    "clf__C": [1],
    "clf__kernel":['linear'],
    "clf__class_weight" : ['balanced'],
    "clf__decision_function_shape":['ovo'],
    "clf__gamma":[0.00001,0.00005,0.0001]
}

gs_svm = GridSearchCV(estimator=pipe_svm,
            param_grid=parameters_svm,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_svm.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_svm.best_score_)

# Best params
print('\nBest params:\n', gs_svm.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  3.2min finished


Best accuracy: 0.687

Best params:
 {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__decision_function_shape': 'ovo', 'clf__gamma': 1e-05, 'clf__kernel': 'linear', 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [14]:
#pipeline for random forest take 2

pipe_rf2 = Pipeline([
     ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])
parameters_rf2 = {
    'tfidf__min_df': [0.001],
    'tfidf__max_df': [1.0],
    'tfidf__ngram_range': [(1, 2)],  #bigrams
    "clf__criterion": ["gini"],
    "clf__max_depth":[150,250,500],
    "clf__min_samples_split" : [40,50,75]
}

gs_rf2 = GridSearchCV(estimator=pipe_rf2,
            param_grid=parameters_rf2,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_rf2.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_rf2.best_score_)

# Best params
print('\nBest params:\n', gs_rf2.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.8min finished


Best accuracy: 0.633

Best params:
 {'clf__criterion': 'gini', 'clf__max_depth': 500, 'clf__min_samples_split': 40, 'tfidf__max_df': 1.0, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 2)}


In [15]:
#pipeline for knn

pipe_knn = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 0.001)),
#     ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   KNeighborsClassifier()),
])

parameters_knn = {
#     'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1,1),(1, 2)],  #  bigrams
#     "svd__n_components": [100,250,500],
    "clf__n_neighbors": [1,3,5,10],
    "clf__metric": ['minkowski', 'euclidean','wminkowski']
}

gs_knn = GridSearchCV(estimator=pipe_knn,
            param_grid=parameters_knn,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_knn.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_knn.best_score_)

# Best params
print('\nBest params:\n', gs_knn.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   57.7s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  2.4min finished


Best accuracy: 0.158

Best params:
 {'clf__metric': 'minkowski', 'clf__n_neighbors': 1, 'tfidf__ngram_range': (1, 1)}


In [16]:
#pipeline for knn

pipe_knn = Pipeline([
    ('tfidf', TfidfVectorizer(min_df = 0.001)),
    ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   KNeighborsClassifier()),
])

parameters_knn = {
#     'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1,1),(1, 2)],  #  bigrams
    "svd__n_components": [100,250,500],
    "clf__n_neighbors": [1,3,5,10],
    "clf__metric": ['minkowski', 'euclidean','wminkowski']
}

gs_knn = GridSearchCV(estimator=pipe_knn,
            param_grid=parameters_knn,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_knn.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_knn.best_score_)

# Best params
print('\nBest params:\n', gs_knn.best_params_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.6min
/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 30.3min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 45.1min finished


Best accuracy: 0.506

Best params:
 {'clf__metric': 'minkowski', 'clf__n_neighbors': 10, 'svd__n_components': 100, 'tfidf__ngram_range': (1, 2)}


In [ ]:
#pipeline for xgboost

pipe_xgb = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=0.001)),
    ('svd',   TruncatedSVD(algorithm='randomized')),
    ('clf',   XGBClassifier(objective='multi:softmax', num_class=20)),
])

parameters_xgb = {
#     'tfidf__min_df': [0.001],
#     'tfidf__max_df': [0.5, 0.75, 1.0],
    'tfidf__ngram_range': [(1,2),(1, 2)],  #  bigrams
    "svd__n_components": [500],
    "clf__n_estimators": [500],
    "clf__learning_rate":[0.01,0.1,1],
    "clf__min_samples_split" : [50,100]
}

gs_xgb = GridSearchCV(estimator=pipe_xgb,
            param_grid=parameters_xgb,
            scoring='accuracy',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model = gs_xgb.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs_xgb.best_score_)

# Best params
print('\nBest params:\n', gs_xgb.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
